In [1]:
from mesa import Agent, Model
from mesa.space import SingleGrid
from mesa.time import RandomActivation
from mesa.datacollection import DataCollector
import random
from enum import Enum

/home/tarun/.pyenv/versions/3.7.0/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


I was curious about how fast the COVID-19 spread all over the world. So I decided to model the spread using the python agent-based modelling library called **mesa**. Mesa already provides an example of [Virus on network](https://github.com/projectmesa/mesa/blob/master/examples/virus_on_network/virus_on_network), which we'll be extending to fit our scenario.

In [2]:
class State(Enum):
    SUSCEPTIBLE = 0
    INFECTED = 1
    RESISTANT = 2
    CURED = 3

In [54]:
#x_max = can i get this from model? 
#y_max = 
def compute_state(agent):
    '''
    Assumptions:
    - Only immediate neighbors can infect
    - One infected in immediate neighbors is equivalent for the central agent as all of them infected
    '''
    if agent.state in [State.RESISTANT or State.CURED]:
        return
    elif agent.state == State.SUSCEPTIBLE:
        # Can get infected from the already infected
        grid_ = agent.model.grid
        inf_prob_single = agent.model.infection_spread_probab
        for nbr in grid_.iter_cell_list_contents(grid_.get_neighborhood(agent.pos, moore=True, include_center=False)):
            if nbr.state == State.INFECTED:
                chance_spread = agent.model.random.random()
                if(chance_spread < inf_prob_single):
                    agent.state = State.INFECTED
                    break   
    else:
        # Can get tested-for, quarantined and cured
        if agent.model.random.random() < agent.model.testing_probab:
            agent.state = State.CURED
        
    
class Person(Agent):
    def __init__(self, unique_id, model, state):
        super().__init__(unique_id, model)
        self.state = state
    
    def move(self):
        possible_destinations = self.model.grid.get_neighborhood(self.pos, moore=True, include_center=False)
        empty_dest = [dest for dest in possible_destinations if self.model.grid.is_cell_empty(dest)]
        new_pos = self.random.choice(empty_dest)
        self.model.grid.move_agent(self, new_pos)
    
    def step(self):
        self.move()

In [47]:
def count_population(model):
    sus, inf, res, cur = 0,0,0,0
    for agent in model.schedule.agents:
        if agent.state == State.SUSCEPTIBLE:
            sus+=1
        elif agent.state == State.INFECTED:
            inf+=1
        elif agent.state == State.RESISTANT:
            res+=1
        else:
            cur+=1
    model.state = [sus, inf, res, cur]
    if inf == 0:
        model.running = False
    return

class Earth(Model):
    """
    Earth as a 2D grid.
    initial_state = [susceptible, infected, resistant, cured]
    These 4 values should sum to N.
    """
    def __init__(self, N, width, height, infection_spread_probab, testing_probab, initial_state):
        self.population = N
        self.width = width
        self.height = height
        self.infection_spread_probab = infection_spread_probab
        self.testing_probab = testing_probab
        self.grid = SingleGrid(width, height, torus=True)
        self.schedule = RandomActivation(self)
        self.running = True
        self.state = initial_state
        self.datacollector = DataCollector(
            model_reporters={'count': count_population},
            agent_reporters={'State': compute_state}
        )
        
        # Set state
        for i in range(self.population):
            s, inf, r, c = self.state
            if 0 <= i < s:
                person_state = State.SUSCEPTIBLE
            elif s <= i < inf:
                person_state = State.INFECTED
            elif inf <= i < r:
                person_state = State.RESISTANT
            else:
                person_state = State.CURED
            person = Person(i, self, person_state)
            self.schedule.add(person)
            
            # place the person somewhere randomly
            pos = (0,0)
            while(not self.grid.is_cell_empty(pos)):
                x = self.random.randrange(self.grid.width)
                y = self.random.randrange(self.grid.height)
                pos = (x, y)
            self.grid.place_agent(person, pos)
    
    def step(self):
        # once everyone is cured, the model stops running
        # assumption: a cured person can't get infected
        self.schedule.step()
        self.datacollector.collect(self)
        print(self.state)

In [58]:
model = Earth(4, 4,4,1,0.2,[3, 1, 0, 0])
# something is going wrong here!
for i in range(10):
    model.step()
    print('----')

[3, 0, 0, 1]
----
[3, 0, 0, 1]
----
[3, 0, 0, 1]
----
[3, 0, 0, 1]
----
[3, 0, 0, 1]
----
[3, 0, 0, 1]
----
[3, 0, 0, 1]
----
[3, 0, 0, 1]
----
[3, 0, 0, 1]
----
[3, 0, 0, 1]
----


TODO: 
1. Visualize and Batch run: https://mesa.readthedocs.io/en/master/tutorials/intro_tutorial.html
2. Improve model compared to https://dmnfarrell.github.io/bioinformatics/abm-mesa-python